# Valutazioni finali
Con questo notebook si conclude il nostro progetto. Trarremo delle considerazione sui risultati ottenuti, approffonendo quali strategia hanno funzionato meglio. 

## Funzioni note
Le seguenti celle contengono le librerie e le funzioni create durante lo sviluppo del progetto. Non sono state apportate sostanziali modifiche, a ragion di ciò non ne andremo a descrivere nuovamente il funzionamento. 

In [1]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import tensorflow as tf
import os 
import scipy.fftpack as scipy
import onnxruntime as rt
import tf2onnx
import onnx
import random

In [2]:
train_ds, validation_ds = tf.keras.utils.audio_dataset_from_directory(
    directory='../reduced_dataset/dataset/audio',
    validation_split=0.4, # stiamo mettendo da parte il 40% del dataset, che sarà suddiviso in validation set e test set
    shuffle=True,
    subset='both', # necessario se stiamo utilizzando validation_split (se no darebbe errore)
    seed=0 # necessario se stiamo utilizzando sia shuffle che validation_split (se no darebbe errore)
)

noise_ds = tf.keras.utils.audio_dataset_from_directory(
    directory='../noise_dataset',
    batch_size = 1
)

noise_label_names = noise_ds.class_names

val_ds = validation_ds.take(validation_ds.cardinality() // 2) # ho cambiato nome del validation_ds in modo tale da non creare problemi con l'istruzione seguente
test_ds = validation_ds.skip(validation_ds.cardinality() // 2)

Found 12933 files belonging to 30 classes.
Using 7760 files for training.
Using 5173 files for validation.
Found 14 files belonging to 14 classes.


In [3]:
def cut_audios(dataset, length):
    new_audios = [] # inizializziamo una lista dove inseriremo i nostri audio tagliati
    labels = []
    
    # iteriamo nel dataset
    for audio, label in dataset:
        # tagliamo l'audio ad un secondo e lo appendiamo a una lista 
        labels.append(label.numpy())
        
        audio = tf.reshape(audio, [-1])[np.shape(audio)[1]//2:np.shape(audio)[1]//2 + length]
        new_audios.append(audio.numpy()) # convertiamo in array per poterli modificare
        
    return new_audios, labels

cut_noise_audios, cut_noise_labels = cut_audios(noise_ds, 16000)

def mix_audios(original_audios, noise_audios):
    mixed_dataset = [] # inizializziamo la lista dove inseriremo gli audio uniti al rumore

    original_audios = original_audios.unbatch() # il nostro training set ha una batch_size di 32, per rendere il processo più semplice unbatchiamo
    
    # per ogni audio del dataset originale
    for audio, label in original_audios:    
        audio = np.squeeze(audio, axis=-1) # rimuoviamo l'ultima asse inutile (quella dei canali)
        
        # Scegliamo in modo randomico un audio dalla lista degli audio rumorosi
        noise_sample = random.choice(noise_audios)

        # calcolo l'ampiezza massima dell'audio rumoroso
        max_amplitude_audio = np.max(np.abs(audio))
        max_amplitude_noise = np.max(np.abs(noise_sample))
        # calcolo un noise factor che varia a seconda dell'ampiezza massima di entrambi gli audio
        noise_factor = max_amplitude_audio / max_amplitude_noise
        noise_factor = min(noise_factor, 1.0)
        
        noise_sample = noise_sample * noise_factor
        
        mixed_audio = audio + noise_sample # uniamo l'audio original al rumore

        # aggiungiamo l'audio con noise alla lista
        mixed_dataset.append((mixed_audio, label))

    return mixed_dataset

mixed_train_list = mix_audios(train_ds, cut_noise_audios)
mixed_val_list = mix_audios(validation_ds, cut_noise_audios)

def create_mixed_ds(dataset_list):
    audio_data = [tf.convert_to_tensor(audio, dtype=tf.float32) for audio, label in dataset_list]
    labels = [label for _, label in dataset_list]

    audio_data = tf.expand_dims(audio_data, axis=-1)
    
    mixed_train_ds = tf.data.Dataset.from_tensor_slices((audio_data, labels))
    mixed_train_ds = mixed_train_ds.batch(32)
    return mixed_train_ds

mixed_train_ds = create_mixed_ds(mixed_train_list)
mixed_validation_ds = create_mixed_ds(mixed_val_list)

mixed_val_ds = mixed_validation_ds.take(mixed_validation_ds.cardinality() // 2) # ho cambiato nome del validation_ds in modo tale da non creare problemi con l'istruzione seguente
mixed_test_ds = mixed_validation_ds.skip(mixed_validation_ds.cardinality() // 2)

In [4]:
class DatasetConverter:
    def __init__(self, dataset):
        self.dataset = dataset

    def convert(self, option):
        available_options = ['spectrogram', 'filterbanks', 'mfcc']
        
        if option == available_options[0]:
            return self.get_spectrogram_dataset()
        elif option == available_options[1]:
            return self.get_filterbanks_dataset()
        elif option == available_options[2]:
            return self.get_mfcc_dataset()
        else:
            raise ValueError(f"Opzione non disponibile: inserire una delle seguenti opzioni: {available_options}")
    
    # INIZIO SPETTROGRAMMI
    def squeeze(self, audio, labels):
        audio = tf.squeeze(audio, axis=-1)
        return audio, labels
    
    def get_spectrogram(self, waveform):
    # applichiamo la short-time Fourier transorm
        spectrogram = tf.signal.stft(waveform, frame_length=255, frame_step=128)
        spectrogram = tf.abs(spectrogram)
        
        return spectrogram[..., tf.newaxis]
    
    def get_spectrogram_dataset(self):
        # squeeze
        self.dataset = self.dataset.map(self.squeeze, tf.data.AUTOTUNE)
        self.dataset = self.dataset.map(lambda x, y: (self.get_spectrogram(x), y), num_parallel_calls=tf.data.AUTOTUNE)
        
        return self.dataset

    # FINE SPETTROGRAMMI

    def convert_to_numpy(self, dataset):
        audio_data = []
        labels = []
    
        dataset = dataset.unbatch()
        
        for audio, label in dataset:
            audio_data.append(audio.numpy())  # Assuming audio is a tensor, convert to numpy array
            labels.append(label.numpy())      # Assuming label is a tensor, convert to numpy array
        
        audio_data = np.array(audio_data)
        labels = np.array(labels)
        
        return audio_data, labels
    
    # INIZIO FILTERBANKS
    def makeHamming(self, M):
        R = (( M - 1 ) / 2 , M / 2)[M % 2 == 0]
        w = (np.hamming(M), np.hamming(M + 1))[M % 2 == 0]
        if M % 2 != 0:
            w[0] = w[0]/2
            w[M-1] = w[M-1]/2
        else:
            w = w[:M]
    
        return w

    def hztomel(self, hz):
        return (2595 * np.log10(1 + hz / 700))

    def meltohz(self, mel):
        return (700 * (10**(mel / 2595) - 1))

    def compute_filterbanks(self, audios_np, pre_emphasis=0.97, sample_rate=16000, frame_size=0.025, frame_stride=0.01, NFFT=512, nfilt=40):
        filterbanks_np = []
        
        for samples in audios_np:
            emphasized_audio = np.append(samples[0], samples[1:] - pre_emphasis * samples[:-1])
            audio_length = len(emphasized_audio)
    
            frame_length, frame_step = int(frame_size * sample_rate), int(frame_stride * sample_rate)
    
            num_frames = int(np.ceil(float(np.abs(audio_length - frame_length)) / frame_step))
    
            pad_audio_length = num_frames * frame_step + frame_length
            z = np.zeros((pad_audio_length - audio_length))
            pad_audio = np.append(emphasized_audio, z)
    
            indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + np.tile(np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
            frames = pad_audio[indices.astype(np.int32, copy=False)]
    
            # Usiamo la funzione di Hamming
            hamming_window = self.makeHamming(frame_length)
    
            mag_frames = np.absolute(np.fft.rfft(frames, NFFT))  # Magnitudo della FFT
            pow_frames = ((1.0 / NFFT) * ((mag_frames) ** 2))
    
            # convertiamo hz in mel
            low_freq_mel = self.hztomel(0)
            high_freq_mel = self.hztomel(sample_rate / 2)
    
            mel_points = np.linspace(low_freq_mel, high_freq_mel, nfilt + 2)
            hz_points = self.meltohz(mel_points) 
    
            bin = np.floor((NFFT + 1) * hz_points / sample_rate)
    
            fbank = np.zeros((nfilt, int(np.floor(NFFT / 2 + 1))))
    
            for m in range(1, nfilt + 1):
                f_m_minus = int(bin[m - 1])
                f_m = int(bin[m])
                f_m_plus = int(bin[m + 1])
    
                for k in range(f_m_minus, f_m):
                    fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
                for k in range(f_m, f_m_plus):
                    fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])
    
            # in questo momento invece calcoliamo i filter banks per i segmenti di audio, utilizzando i filtri triangolari appena creati
            filter_banks = np.dot(pow_frames, fbank.T)
            filter_banks = np.where(filter_banks == 0, np.finfo(float).eps, filter_banks)
            filter_banks = 20 * np.log10(filter_banks)
    
            filterbanks_np.append(filter_banks)
        
        return np.array(filterbanks_np)
    
    def get_filterbanks_dataset(self): 
        audios, labels = self.convert_to_numpy(self.dataset)

        filterbanks = self.compute_filterbanks(audios)
        filterbanks = np.expand_dims(filterbanks, axis=-1)

        self.dataset = tf.data.Dataset.from_tensor_slices((filterbanks, labels))
        self.dataset = self.dataset.batch(32)
        
        return self.dataset
    # FINE FILTERBANKS

    # INIZIO MFCC
    def compute_mfcc(self, filter_banks, num_ceps=12, cep_lifter=22):
        mfcc_np = []
        
        for f in filter_banks:
            mfcc = scipy.dct(f, type=2, axis=1, norm='ortho')[:, 1 : (num_ceps + 1)]

            (nframes, ncoeff) = mfcc.shape
            n = np.arange(ncoeff)
            
            lift = 1 + (cep_lifter / 2) * np.sin(np.pi * n / cep_lifter)
            mfcc *= lift

            mfcc_np.append(mfcc)
        
        return np.array(mfcc_np)
    
    
    def get_mfcc_dataset(self):
        audios, labels = self.convert_to_numpy(self.dataset)

        filterbanks = self.compute_filterbanks(audios)
        mfcc = self.compute_mfcc(filterbanks)
        
        mfcc = np.expand_dims(mfcc, axis=-1)
        
        self.dataset = tf.data.Dataset.from_tensor_slices((mfcc, labels))
        self.dataset = self.dataset.batch(32)
        
        return self.dataset
    
    # FINE MFCC

## Funzioni nuove

Per poter rappresentare i dati ottenuti abbiamo dovuto modificare `evaluate_onnx_model`. Se fino ad ora questa funzione aveva solamente la funzione di printare le metriche di valutazioni migliori, ora restituisce direttamente l'accuratezza e la perdita. 

In [5]:
def evaluate_onnx_model(path_model_onnx, test_ds):
    # il fatto che sia suddiviso in batch mi crea problemi, perciò lo risolvo togliendoli
    test_ds = test_ds.unbatch()
    
    # carico il modello utilizzando il file onnx
    m = rt.InferenceSession(path_model_onnx)
    
    # trasformo il dataset in array numpy
    spectrogram_np = np.array([spectrogram.numpy() for spectrogram, _ in test_ds], dtype=np.float32)
    labels_np = np.array([label.numpy() for _, label in test_ds])
    
    # eseguo le predizione del modello
    pred_onnx = m.run(None, {'input': spectrogram_np})
    # ottengo la predizione corretta
    predictions = np.argmax(pred_onnx[0], axis=1)
    # computo la accuratezza
    accuracy = np.mean(predictions == labels_np)
    # computo la loss
    sparse_categorical_loss = tf.keras.losses.sparse_categorical_crossentropy(labels_np, pred_onnx[0])
    mean_sparse_categorical_loss = np.mean(sparse_categorical_loss)
    # stampo l'accuratezza
    return round(accuracy, 3), round(mean_sparse_categorical_loss, 3)

Per poter analizzare meglio le reti create è stata sviluppata la funzione `create_table`, la quale crea una struttura DataFrame di Pandas in base al dataset e all'ottimizzatore passati come argomenti e alla scelta di avere o meno il dataset rumoroso. Imposta di default come ottimizzatore **rmsprop** e sceglie il dataset senza rumore.

In [6]:
# creiamo le colonne e il dataframe
headers = ["Modello", "Opt", "Dataset", "Rumore", "Accuratezza", "Perdita"]

def create_table(dataset_name, optimizer="rmsprop", noise=False):
    if noise:
        dataset = DatasetConverter(mixed_test_ds)
        dataset = dataset.convert(dataset_name)
    else:
        dataset = DatasetConverter(test_ds)
        dataset = dataset.convert(dataset_name)
        
    path = f"bestmodels/noise/{optimizer}/" if noise else f"bestmodels/{optimizer}/"
    path += dataset_name + "/"

    df = pd.DataFrame(columns=headers)

    for model in os.listdir(path):
        if model.endswith("onnx"):
            new_row = []
            model_name = model.split(".onnx")[0]
            model_name = model_name.split("_")
            
            if model_name[-1] != "model":
                model_name = model_name[:-1]

            model_name = "_".join(model_name)
  
            new_row.append(model_name)

            new_row.append(optimizer)
            
            new_row.append(path.split("/")[-2])
            
            if noise:
                new_row.append("Sì")
            else:
                new_row.append("No")

            accuracy, loss = evaluate_onnx_model(path+model, dataset)
            new_row.append(accuracy)
            new_row.append(loss)

            df.loc[-1] = new_row
            df.index = df.index + 1

    return df

## Dataset originale

### Valutazione spettrogrammi
Partiamo da analizzare i risultati dei modelli con il primo tipo di dati creati a partire dal nostro dataset di audio di partenza. 

Per farlo abbiamo bisogno di creare due dataframe, uno per l'ottimizzatore rmsprop e un altro per l'adam, che uniremo per creare una tabella unica. 

In [7]:
df_spect_rmsprop = create_table("spectrogram")
df_spect_adam = create_table("spectrogram", optimizer="adam")

# Aggiunta resnet
spect_test_ds = DatasetConverter(test_ds)
spect_test_ds = spect_test_ds.convert("spectrogram")

acc, loss = evaluate_onnx_model("bestmodels/resnet/voxprofunda_spect.onnx", spect_test_ds)
new_row = ["voxprofunda", "adam", "spectrogram", "No", acc, loss]

df_spect_adam.loc[-1] = new_row; df_spect_adam.index = df_spect_adam.index + 1
# fine aggiunta resnet

df_spect = pd.concat([df_spect_rmsprop, df_spect_adam]).sort_values(by="Accuratezza", ascending=False).reset_index(drop=True)

df_spect

,Modello,Opt,Dataset,Rumore,Accuratezza,Perdita
0,voxprofunda,adam,spectrogram,No,0.932,0.326
1,bp_basic_model,adam,spectrogram,No,0.897,0.448
2,bp_basic_model,rmsprop,spectrogram,No,0.891,0.437
3,improved_basic_model,rmsprop,spectrogram,No,0.863,0.549
4,improved_basic_model,adam,spectrogram,No,0.855,0.649
5,basic_model,rmsprop,spectrogram,No,0.828,0.804
6,basic_model,adam,spectrogram,No,0.788,0.788
7,tuned_improved_basic_model,rmsprop,spectrogram,No,0.775,1.066
8,tuned_improved_basic_model,adam,spectrogram,No,0.769,1.145


Da questa tabella possiamo notare come le differenze tra i due ottimizzatori non siano marcate, ma in quasi tutti i casi **rmsprop** per questo tipo di dati ha performato meglio. 

Esaminando nello specifico i singoli modelli si può osservare come la rete convolutivo base partisse con un buon valore di accuratezza, intorno all'**80%**, ma aggiungendo un semplice livello di dropout questa metrica si è attesta sull'**85%**.

I risultati migliori sono stati ottenuti dall'adozione delle best practice. L'impiego di blocchi residui ha fatto sì che il modello raggiungesse un risultato di quasi **90%** di accuratezza, ben **10** punti percentuali in più rispetto al primo convolutivo.

Come ci aspettavamo, l'architettura VGGVox è quella che ha portato risultati migliori, superando il **93%**. Detto ciò bisogna considerare anche il numero elevato di parametri, circa **23.5M**, mentre la rete convolutiva con i blocchi residui ne ha poco più del **0.7%**. 

Scarsi i risultati ottenuti dalla tecnica di tuning degli iperparametri, dove si è raggiunta l'accuratezza più bassa tra tutti i modelli. Questo probabilmente dovuto al fatto che è stato impiegato un tasso di apprendimento fisso, mentre per gli altri modelli sono stati usati dei tassi di apprendimento dinamici, i quali si adattano ad ogni epoca. 

## Valutazione filterbank

In [8]:
df_fb_rmsprop = create_table("filterbanks")
df_fb_adam = create_table("filterbanks", optimizer="adam")

# Aggiunta resnet
fb_test_ds = DatasetConverter(test_ds)
fb_test_ds = fb_test_ds.convert("filterbanks")

acc, loss = evaluate_onnx_model("bestmodels/resnet/voxprofunda_fb.onnx", fb_test_ds)
new_row = ["voxprofunda", "adam", "filterbanks", "No", acc, loss]

df_fb_adam.loc[-1] = new_row; df_fb_adam.index = df_fb_adam.index + 1
# fine aggiunta resnet

df_fb = pd.concat([df_fb_rmsprop, df_fb_adam]).sort_values(by="Accuratezza", ascending=False).reset_index(drop=True)

df_fb

,Modello,Opt,Dataset,Rumore,Accuratezza,Perdita
0,voxprofunda,adam,filterbanks,No,0.917,0.346
1,improved_basic_model,rmsprop,filterbanks,No,0.881,0.532
2,bp_basic_model,adam,filterbanks,No,0.881,0.531
3,bp_basic_model,rmsprop,filterbanks,No,0.871,0.607
4,tuned_improved_basic_model,adam,filterbanks,No,0.848,0.598
5,tuned_improved_basic_model,rmsprop,filterbanks,No,0.846,0.601
6,improved_basic_model,adam,filterbanks,No,0.843,0.579
7,basic_model,adam,filterbanks,No,0.831,0.617
8,basic_model,rmsprop,filterbanks,No,0.824,0.659


Sfruttando i filterbank i modelli più semplici hanno performato meglio rispetto agli spettrogrammi e le differenze tra gli ottimizzatori sono più rilevanti. In linea generale per questo tipo di dati quello ad avere la meglio è stato l'**adam**. 

Il modello convolutivo di partenza ha performato in modo smile a quello usato per gli spettrogrammi, infatti raggiunge un'accuratezza dell'**82%** con ottimizzare **rmsprop**. Interessante notare come dopo aver aggiunto lo strato di dropout la rete ottiene un miglioramento notevole con **88%**, ma lo stesso risultato non avviene per l'ottimizzatore **adam**, il quale migliora di appena **1** punto percentuale.  

I modelli a cui è stato effettuato il tuning degli iperparametri questa volta hanno apportato dei miglioramenti rispetto all'architettura base, anche se solo di qualche punto percentile.

Anche in questo specifico caso i modelli a cui sono state effettuate le best practice hanno ottenuto i benefici migliori salendo a **88%** e **87%** di accuracy, rispettivamente per **adam** e **rmsprop**. 

L'archiettura VGGVox ha performato meglio di tutte, ma peggio rispetto alla corrispettiva impiegata con il dataset di spettrogrammi.

## Valutazione MFCC

In [9]:
df_mfcc_rmsprop = create_table("mfcc")
df_mfcc_adam = create_table("mfcc", optimizer="adam")

# Aggiunta resnet
mfcc_test_ds = DatasetConverter(test_ds)
mfcc_test_ds = mfcc_test_ds.convert("mfcc")

acc, loss = evaluate_onnx_model("bestmodels/resnet/voxprofunda_mfcc.onnx", mfcc_test_ds)
new_row = ["voxprofunda", "adam", "mfcc", "No", acc, loss]

df_mfcc_adam.loc[-1] = new_row; df_mfcc_adam.index = df_mfcc_adam.index + 1
# fine aggiunta resnet

df_mfcc = pd.concat([df_mfcc_rmsprop, df_mfcc_adam]).sort_values(by="Accuratezza", ascending=False).reset_index(drop=True)

df_mfcc

,Modello,Opt,Dataset,Rumore,Accuratezza,Perdita
0,voxprofunda,adam,mfcc,No,0.882,0.400
1,bp_basic_model,adam,mfcc,No,0.831,0.740
2,improved_basic_model,rmsprop,mfcc,No,0.814,0.786
3,basic_model,rmsprop,mfcc,No,0.797,0.850
4,bp_basic_model,rmsprop,mfcc,No,0.796,0.795
5,improved_basic_model,adam,mfcc,No,0.780,0.875
6,basic_model,adam,mfcc,No,0.742,0.968
7,tuned_improved_basic_model,adam,mfcc,No,0.683,1.146
8,tuned_improved_basic_model,rmsprop,mfcc,No,0.681,1.147


I tipi di dati MFCC sono stati quelli che hanno messo più in difficoltà i modelli. Quelli a cui è stato impostato come ottimizzatore rmsprop hanno performato meglio. 

La rete di partenza con l'ottimizzatore migliore ha raggiunto performance simili a quelle della stessa allenata con gli altri tipi di dati e le migliorie apportate non hanno condotto a risultati significativi, andando anche a peggiorare con l'impiego dei blocchi residui. 

La stessa rete in cui è stato impostato l'ottimizzatore peggiore ha seguito un andamento simile ribasatto di qualche punto, eccezion fatta per quella che impiega le best practice che è riuscita a performare meglio di tutte. 

Il modello tuning non è stato efficace in nessun caso, ottenendo come valori di accuracy **68%** e di perdira sopra l'**1**. 

Il modello Voxprofunda continua a eccellere con un'accuratezza di **88%**.

## Dataset + noise

### Valutazione spettrogrammi

In [11]:
df_spect_rmsprop_noise = create_table("spectrogram", noise=True)
df_spect_adam_noise = create_table("spectrogram", optimizer="adam", noise=True)

# Aggiunta resnet
spect_mixed_test_ds = DatasetConverter(mixed_test_ds)
spect_mixed_test_ds = spect_mixed_test_ds.convert("spectrogram")

acc, loss = evaluate_onnx_model("bestmodels/noise/resnet/voxprofunda_spect.onnx", spect_mixed_test_ds)
new_row = ["voxprofunda", "adam", "spectrogram", "Sì", acc, loss]

df_spect_adam_noise.loc[-1] = new_row; df_spect_adam_noise.index = df_spect_adam_noise.index + 1
# fine aggiunta resnet

df_spect_noise = pd.concat([df_spect_rmsprop_noise, df_spect_adam_noise]).sort_values(by="Accuratezza", ascending=False).reset_index(drop=True)

df_spect_noise

,Modello,Opt,Dataset,Rumore,Accuratezza,Perdita
0,voxprofunda,adam,spectrogram,Sì,0.812,0.693
1,tuned_improved_basic_model,rmsprop,spectrogram,Sì,0.710,1.296
2,improved_basic_model,rmsprop,spectrogram,Sì,0.703,1.058
3,tuned_improved_basic_model,adam,spectrogram,Sì,0.680,1.356
4,improved_basic_model,adam,spectrogram,Sì,0.662,1.152
5,basic_model,adam,spectrogram,Sì,0.657,1.283
6,basic_model,rmsprop,spectrogram,Sì,0.653,1.219
7,bp_basic_model,rmsprop,spectrogram,Sì,0.644,1.514
8,bp_basic_model,adam,spectrogram,Sì,0.599,1.413


Come era facile intuire, aggiungendo del rumore agli audio originali le prestazioni generali sono peggiorate. I modelli con l'ottimizzatore adam, eccetto Voxprofunda, tendono ad avere accuratezze inferiori e perdite più alte, indicando che potrebbero non essere ottimizzati al meglio per questo specifico compito.

Il Tuned Improved Basic Model e l'Improved Basic Model seguono, ma con **10** punti percentuale in meno di accuratezza e presentano perdite significativamente più alte, suggerendo possibili difficoltà di convergenza.  

I modelli convolutivi base e bp basic mostrano basse performance, sia per accuratezza che per perdita, suggerendo problemi di sovra-allenamento o difficoltà a gestire il rumore. 

### Valutazione filterbank

In [12]:
df_fb_rmsprop_noise = create_table("filterbanks", noise=True)
df_fb_adam_noise = create_table("filterbanks", optimizer="adam", noise=True)

# Aggiunta resnet
fb_mixed_test_ds = DatasetConverter(mixed_test_ds)
fb_mixed_test_ds = fb_mixed_test_ds.convert("filterbanks")

acc, loss = evaluate_onnx_model("bestmodels/noise/resnet/voxprofunda_fb.onnx", fb_mixed_test_ds)
new_row = ["voxprofunda", "adam", "filterbanks", "Sì", acc, loss]

df_fb_adam_noise.loc[-1] = new_row; df_fb_adam_noise.index = df_fb_adam_noise.index + 1
# fine aggiunta resnet

df_fb_noise = pd.concat([df_fb_rmsprop_noise, df_fb_adam_noise]).sort_values(by="Accuratezza", ascending=False).reset_index(drop=True)

df_fb_noise

,Modello,Opt,Dataset,Rumore,Accuratezza,Perdita
0,voxprofunda,adam,filterbanks,Sì,0.795,0.756
1,improved_basic_model,rmsprop,filterbanks,Sì,0.759,0.952
2,tuned_improved_basic_model,rmsprop,filterbanks,Sì,0.739,1.056
3,improved_basic_model,adam,filterbanks,Sì,0.727,1.036
4,tuned_improved_basic_model,adam,filterbanks,Sì,0.715,0.963
5,basic_model,rmsprop,filterbanks,Sì,0.713,1.244
6,basic_model,adam,filterbanks,Sì,0.665,1.327
7,bp_basic_model,rmsprop,filterbanks,Sì,0.624,1.535
8,bp_basic_model,adam,filterbanks,Sì,0.594,1.363


Nella tabella dei filterbanks, il modello Voxprofunda si distingue con la più alta accuratezza **79.5** e una perdita relativamente bassa. 

L'Improved Basic Model segue con un'accuratezza di **75.9**, mostrando una performance solida, ma inferiore rispetto al VGGVox. Il Tuned Improved Basic Model con **rmsprop** e l'Improved Basic Model con **adam** presentano accuratezze leggermente inferiori, rispettivamente **73.9** e **72.7**.

I modelli Basic e BP Basic mostrano performance variabili, con accuratezze che vanno da 0.713 a 0.594 e perdite che vanno da 1.244 a 1.535, suggerendo una maggiore difficoltà nel gestire il rumore.

### Valutazione MFCC

In [13]:
df_mfcc_rmsprop_noise = create_table("mfcc", noise=True)
df_mfcc_adam_noise = create_table("mfcc", optimizer="adam", noise=True)

# Aggiunta resnet
mfcc_mixed_test_ds = DatasetConverter(mixed_test_ds)
mfcc_mixed_test_ds = mfcc_mixed_test_ds.convert("mfcc")

acc, loss = evaluate_onnx_model("bestmodels/noise/resnet/voxprofunda_mfcc.onnx", mfcc_mixed_test_ds)
new_row = ["voxprofunda", "adam", "mfcc", "Sì", acc, loss]

df_mfcc_adam_noise.loc[-1] = new_row; df_mfcc_adam_noise.index = df_mfcc_adam_noise.index + 1
# fine aggiunta resnet

df_mfcc_noise = pd.concat([df_mfcc_rmsprop_noise, df_mfcc_adam_noise]).sort_values(by="Accuratezza", ascending=False).reset_index(drop=True)

df_mfcc_noise

,Modello,Opt,Dataset,Rumore,Accuratezza,Perdita
0,voxprofunda,adam,mfcc,Sì,0.720,0.978
1,improved_basic_model,rmsprop,mfcc,Sì,0.632,1.388
2,bp_basic_model,rmsprop,mfcc,Sì,0.625,1.342
3,improved_basic_model,adam,mfcc,Sì,0.621,1.451
4,basic_model,rmsprop,mfcc,Sì,0.612,1.539
5,bp_basic_model,adam,mfcc,Sì,0.594,1.383
6,tuned_improved_basic_model,rmsprop,mfcc,Sì,0.581,1.614
7,tuned_improved_basic_model,adam,mfcc,Sì,0.573,1.501
8,basic_model,adam,mfcc,Sì,0.510,1.678


Per quanto riguarda i MFCC, il modello Voxprofunda mantiene il primato, ma con dei risultati decisamente inferiori, con un'accuratezza di **72%**.

L'Improved Basic Model **rmsprop** segue con un'accuratezza di **63%**.

Gli altri modelli, inclusi il Basic Model e il Tuned Improved Basic Model, mostrano performance inferiori, con accuratezze che variano da 0.612 a 0.510. 

## Conclusioni

Dall'analisi delle tabelle sui vari dataset (spettrogrammi, filterbanks e MFCC) con e senza rumore, emergono alcune considerazioni chiave riguardo le performance dei modelli di riconoscimento vocale:

- Voxprofunda, si è dimostrato il più performante in quasi tutti i contesti analizzati, sia in presenza che in assenza di rumore. La sua capacità di mantenere alta accuratezza e bassa perdita in condizioni diverse indica una robustezza superiore e una buona generalizzazione.

- L'aggiunta di rumore tende a peggiorare le performance di tutti i modelli, come ci si aspetterebbe. Tuttavia, Voxprofunda riesce a mantenere una superiorità relativa, suggerendo che sia meglio equipaggiato per gestire dataset rumorosi rispetto agli altri modelli.

- I modelli che utilizzano l'ottimizzatore adam generalmente mostrano performance leggermente inferiori rispetto agli equivalenti con rmsprop. Questo è evidente sia nei dataset con rumore che senza rumore.

- I modelli migliorati con i blocchi residui hanno mostrato performance migliori soprattuto per quanto riguarda il dataset senza rumore e sembrano lavorare meglio con l'ottimizzatore adam.

- I modelli migliorati con lo strato di dropout hanno portato benefici soprattuto quando si tratta il dataset senza rumore e in combinazione all'ottimizzatore rmsprop

- I modelli mostrano variazioni di performance a seconda della rappresentazione utilizzata (spettrogrammi, filterbanks, MFCC). In generale, le performance sono più alte nei dataset senza rumore, con differenze minori tra le rappresentazioni quando il rumore è presente.

- Ulteriori miglioramenti potrebbero essere ottenuti esplorando nuove architetture o ulteriori tecniche di pre-elaborazione e ottimizzazione.